In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_union, Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.utils import resample
from sklearn.compose import ColumnTransformer

### Columns

#### train.csv and test.csv

- Patient- a unique Id for each patient (also the name of the patient's DICOM folder)
- Weeks- the relative number of weeks pre/post the baseline CT (may be negative)
- FVC - the recorded lung capacity in ml
- Percent- a computed field which approximates the patient's FVC as a percent of the typical FVC for a person of similar characteristics
- Age
- Sex
- SmokingStatus

Percent is the comparison of the patient's measured FVC against expected

In [2]:
DATA_DIR = "../tmp/osic-pulmonary-fibrosis-progression" # Local

In [3]:
# Load test and training data
train_df = pd.read_csv(DATA_DIR + '/train.csv') # Use first
test_df = pd.read_csv(DATA_DIR + '/test.csv') # Save for later

# Load exported autoencoder encoding from local pickle
PICKLE_PATH = "patient_ids_to_encodings_dict.pkl"
with open(PICKLE_PATH, "rb") as f:
    patient_ct_encodings = pickle.load(f)

# Convert to DF with patient ID as rows, encoding as 128 columns
patient_ct_encodings = pd.DataFrame.from_dict(patient_ct_encodings, orient="index")

# Left join encoding to training & test data
train_df = train_df.join(patient_ct_encodings, on="Patient", how="left")
test_df = test_df.join(patient_ct_encodings, on="Patient", how="left")

train_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,0,1,2,...,118,119,120,121,122,123,124,125,126,127
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker,0.036863,0.250262,0.214101,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker,0.036863,0.250262,0.214101,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker,0.036863,0.250262,0.214101,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker,0.036863,0.250262,0.214101,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker,0.036863,0.250262,0.214101,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935


In [4]:
test_df.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus,0,1,2,...,118,119,120,121,122,123,124,125,126,127
0,ID00419637202311204720264,6,3020,70.186855,73,Male,Ex-smoker,0.077481,0.273456,0.218755,...,0.060672,0.0,0.006664,0.226692,0.298766,0.0,0.028100,0.0,0.130775,0.0
1,ID00421637202311550012437,15,2739,82.045291,68,Male,Ex-smoker,0.088297,0.298811,0.270244,...,0.090454,0.0,0.019464,0.220450,0.358317,0.0,0.034138,0.0,0.138857,0.0
2,ID00422637202311677017371,6,1930,76.672493,73,Male,Ex-smoker,0.080014,0.270827,0.238446,...,0.062583,0.0,0.022374,0.207912,0.305052,0.0,0.034256,0.0,0.115796,0.0
3,ID00423637202312137826377,17,3294,79.258903,72,Male,Ex-smoker,0.086455,0.285700,0.254137,...,0.089488,0.0,0.024379,0.230229,0.359478,0.0,0.031867,0.0,0.138221,0.0
4,ID00426637202313170790466,0,2925,71.824968,73,Male,Never smoked,0.084378,0.284071,0.253392,...,0.088711,0.0,0.026607,0.225449,0.356416,0.0,0.031750,0.0,0.145091,0.0


In [5]:
# Define custom transformers
class DataFrameSelector(BaseEstimator, TransformerMixin):
    """Selects columns from a Pandas DataFrame using attr"""
    def __init__(self, attr: list):
        self.attr = attr
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attr]


In [6]:
# Make labels
labels_df = train_df[["FVC", "Patient"]]

# Define input classes
# No-op weeks because time progress is significant?
no_op_attrs = ["Weeks"]
num_attrs = ["Percent", "Age"]
cat_attrs = ["Sex", "SmokingStatus"]
# After the join, the encoded CT columns have integer column labels
encoded_attrs = list(range(0, 128))

# Define no-operation pipeline
no_op_pipeline = Pipeline([
    ('selector', DataFrameSelector(no_op_attrs)),
])

# Define numerical pipeline
num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attrs)),
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

# Define categorical pipeline
cat_pipeline = Pipeline([
    ('selector', DataFrameSelector(cat_attrs)),
    ('one_hot_encoder', OneHotEncoder()),
])

# Define encoded CT pipeline
encoded_pipeline = Pipeline([
    ('selector', DataFrameSelector(encoded_attrs)),
    ('imputer', SimpleImputer(strategy="median")),
])

cleaning_pipeline = make_union(no_op_pipeline, num_pipeline, cat_pipeline, encoded_pipeline)


In [7]:
# Prepare training data & convert to DataFrame
X, y = cleaning_pipeline.fit_transform(train_df), train_df[['FVC']]
X = pd.DataFrame(X.todense())

# Reshape y
n_samples = len(X)
y = y.values.reshape(n_samples, )

X.head()

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,-4.0,-0.979923,1.674174,0.0,1.0,0.0,1.0,0.0,0.036863,0.250262,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935
1,5.0,-1.108174,1.674174,0.0,1.0,0.0,1.0,0.0,0.036863,0.250262,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935
2,7.0,-1.302454,1.674174,0.0,1.0,0.0,1.0,0.0,0.036863,0.250262,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935
3,9.0,-1.197060,1.674174,0.0,1.0,0.0,1.0,0.0,0.036863,0.250262,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935
4,11.0,-1.292296,1.674174,0.0,1.0,0.0,1.0,0.0,0.036863,0.250262,...,0.103108,0.0,0.003177,0.183595,0.307978,0.0,0.050988,0.0,0.141652,0.010935


Looking at Bayesian regressors because they have ways to estimate confidence intervals of predictions directly

In [8]:
# Bayesian Ridge Regression
br_regr = BayesianRidge()
br_regr.fit(X.values, y)
y_pred = br_regr.predict(X.values)

print("Bayesian Ridge Regression Training Metrics")
print(f"R2 Score {r2_score(y, y_pred)} \nMAE {mean_absolute_error(y, y_pred)} ")
print(f"First 5 predictions: {y_pred[:4]}")
print(f"First 5 real FVC: {y[:4]}")

Bayesian Ridge Regression Training Metrics
R2 Score 0.9222945407321941 
MAE 181.54737067080262 
First 5 predictions: [2250.4292708  2158.61811322 2030.57005404 2097.10855769]
First 5 real FVC: [2315 2214 2061 2144]


In [9]:
# Decision Tree Regression
tree_regr = DecisionTreeRegressor()
tree_regr.fit(X.values, y)
y_pred = tree_regr.predict(X.values)

print("Decision Tree Regression Training Metrics")
print(f"R2 Score {r2_score(y, y_pred)} \nMAE {mean_absolute_error(y, y_pred)} ")
print(f"First 5 predictions: {y_pred[:4]}")
print(f"First 5 real FVC: {y[:4]}")

Decision Tree Regression Training Metrics
R2 Score 1.0 
MAE 0.0 
First 5 predictions: [2315. 2214. 2061. 2144.]
First 5 real FVC: [2315 2214 2061 2144]


In [12]:
# Random Forest Regression
rf_regr = RandomForestRegressor()
rf_regr.fit(X.values, y)
y_pred = rf_regr.predict(X.values)

print("Random Forest Regression Training Metrics")
print(f"R2 Score {r2_score(y, y_pred)} \nMAE {mean_absolute_error(y, y_pred)} ")
print(f"First 5 predictions: {y_pred[:4]}")
print(f"First 5 real FVC: {y[:4]}")

Random Forest Regression Training Metrics
R2 Score 0.9981590044327435 
MAE 18.03612653324726 
First 5 predictions: [2264.96 2201.01 2074.42 2132.55]
First 5 real FVC: [2315 2214 2061 2144]


In [26]:
# Training data looking good, lets try cross validation
def display_scores(scores, linebreak=False):
    print("MAEs: ", scores)
    print("Mean: ", scores.mean())
    print("Standard Deviation: ", scores.std())
    if linebreak: print("")
        

br_scores = cross_val_score(
    br_regr, 
    X.values, 
    y, 
    scoring="neg_mean_squared_error", # Needs utility function
    cv=10
)
br_scores = np.sqrt(-br_scores) # Get MAE

tree_scores = cross_val_score(
    tree_regr, 
    X.values, 
    y, 
    scoring="neg_mean_squared_error", # Needs utility function
    cv=10
)
tree_scores = np.sqrt(-tree_scores) # Get MAE

rf_scores = cross_val_score(
    rf_regr, 
    X.values, 
    y, 
    scoring="neg_mean_squared_error", # Needs utility function
    cv=10
)
rf_scores = np.sqrt(-rf_scores) # Get MAE


print("Bayesian Ridge: ")
display_scores(br_scores, linebreak=True)
print("Decision Tree: ")
display_scores(tree_scores, linebreak=True)
print("Random Forest: ")
display_scores(rf_scores, linebreak=True)

Bayesian Ridge: 
Scores:  [437.38610023 403.87556605 768.49803962 590.43287672 584.14823925
 449.70600476 549.66126139 523.6234897  452.43489481 481.5163127 ]
Mean:  524.1282785231596
Standard Deviation:  101.41340132256363

Decision Tree: 
Scores:  [408.41549053 408.48576396 330.04232378 439.82765686 505.25465314
 553.9528395  374.87659905 390.0093051  298.4663919  557.35085944]
Mean:  426.6681883254848
Standard Deviation:  83.72013455225901

Random Forest: 
Scores:  [294.88461372 409.09245106 276.92904038 401.52207244 329.77065041
 483.08582466 329.58170888 354.89208386 208.71394174 362.40146366]
Mean:  345.0873850807727
Standard Deviation:  72.77336310696025



### TODO: 
- Grid Search on the hyperparams
- Break out validation test set

In [14]:
# Prepare test data & convert to DataFrame
test_X, test_y = cleaning_pipeline.transform(test_df), test_df[['FVC']]
test_X = pd.DataFrame(test_X.todense())

# Reshape y
n_samples = len(test_X)
test_y = test_y.values.reshape(n_samples, )

test_X.head()

,0,1,2,3,4,5,6,7,8,9,...,126,127,128,129,130,131,132,133,134,135
0,6.0,-0.377749,0.823727,0.0,1.0,0.0,1.0,0.0,0.077481,0.273456,...,0.060672,0.0,0.006664,0.226692,0.298766,0.0,0.028100,0.0,0.130775,0.0
1,15.0,0.220652,0.115022,0.0,1.0,0.0,1.0,0.0,0.088297,0.298811,...,0.090454,0.0,0.019464,0.220450,0.358317,0.0,0.034138,0.0,0.138857,0.0
2,6.0,-0.050470,0.823727,0.0,1.0,0.0,1.0,0.0,0.080014,0.270827,...,0.062583,0.0,0.022374,0.207912,0.305052,0.0,0.034256,0.0,0.115796,0.0
3,17.0,0.080045,0.681986,0.0,1.0,0.0,1.0,0.0,0.086455,0.285700,...,0.089488,0.0,0.024379,0.230229,0.359478,0.0,0.031867,0.0,0.138221,0.0
4,0.0,-0.295086,0.823727,0.0,1.0,0.0,0.0,1.0,0.084378,0.284071,...,0.088711,0.0,0.026607,0.225449,0.356416,0.0,0.031750,0.0,0.145091,0.0


In [16]:
# Calculate test accuracy of trained regressors
br_test_y_pred = br_regr.predict(test_X.values)
tree_test_y_pred = tree_regr.predict(test_X.values)
rf_test_y_pred = rf_regr.predict(test_X.values)

print("Bayesian Ridge Regression Test Metrics")
print(f"R2 Score {r2_score(test_y, br_test_y_pred)} \nMAE {mean_absolute_error(test_y, br_test_y_pred)} ")
print(f"First 5 predictions: {br_test_y_pred[:4]}")
print(f"First 5 real FVC: {test_y[:4]}")

print("\n* * *\n")

print("Decision Tree Regression Test Metrics")
print(f"R2 Score {r2_score(test_y, tree_test_y_pred)} \nMAE {mean_absolute_error(test_y, tree_test_y_pred)} ")
print(f"First 5 predictions: {tree_test_y_pred[:4]}")
print(f"First 5 real FVC: {test_y[:4]}")

print("\n* * *\n")

print("Random Forest Regression Test Metrics")
print(f"R2 Score {r2_score(test_y, rf_test_y_pred)} \nMAE {mean_absolute_error(test_y, rf_test_y_pred)} ")
print(f"First 5 predictions: {rf_test_y_pred[:4]}")
print(f"First 5 real FVC: {test_y[:4]}")

Bayesian Ridge Regression Test Metrics
R2 Score 0.5313200613890208 
MAE 267.77993832962267 
First 5 predictions: [3085.80319736 3016.84036234 2328.14432567 2793.25489541]
First 5 real FVC: [3020 2739 1930 3294]

* * *

Decision Tree Regression Test Metrics
R2 Score 1.0 
MAE 0.0 
First 5 predictions: [3020. 2739. 1930. 3294.]
First 5 real FVC: [3020 2739 1930 3294]

* * *

Random Forest Regression Test Metrics
R2 Score 0.9909961471025198 
MAE 29.776000000000067 
First 5 predictions: [2949.64 2744.9  1932.11 3226.1 ]
First 5 real FVC: [3020 2739 1930 3294]


In [27]:
# Test-data cross validation - only 5 total samples, so use cv_size = 3

br_test_scores = cross_val_score(
    br_regr, 
    test_X.values, 
    test_y, 
    scoring="neg_mean_squared_error", # Needs utility function
    cv=3
)
br_test_scores = np.sqrt(-br_test_scores) # Get MAE

tree_test_scores = cross_val_score(
    tree_regr, 
    test_X.values, 
    test_y, 
    scoring="neg_mean_squared_error", # Needs utility function
    cv=3
)
tree_test_scores = np.sqrt(-tree_test_scores) # Get MAE

rf_test_scores = cross_val_score(
    rf_regr, 
    test_X.values, 
    test_y, 
    scoring="neg_mean_squared_error", # Needs utility function
    cv=3
)
rf_test_scores = np.sqrt(-rf_test_scores) # Get MAE


print("Bayesian Ridge: ")
display_scores(br_test_scores, linebreak=True)
print("Decision Tree: ")
display_scores(tree_test_scores, linebreak=True)
print("Random Forest: ")
display_scores(rf_test_scores, linebreak=True)

Bayesian Ridge: 
Scores:  [215.32326109 766.71219636 395.4857231 ]
Mean:  459.17372684978426
Standard Deviation:  229.56416800843022

Decision Tree: 
Scores:  [864.90606426 813.7140161  186.        ]
Mean:  621.5400267883696
Standard Deviation:  308.68159717295447

Random Forest: 
Scores:  [322.89265871 760.50935895  65.72      ]
Mean:  383.04067255382785
Standard Deviation:  286.81748087621486

